# TP Suivi des blessures


## Récolte des données

In [1]:
#Instalation de la bibliothèque Selenium
!pip install selenium


[notice] A new release of pip available: 22.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# import de l'objet webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome

In [9]:
driver = Chrome()

In [10]:
driver.get("https://www.cbssports.com/nfl/injuries/daily/")

In [11]:
# reject cookies
driver.find_element(By.ID, "onetrust-reject-all-handler").click()


In [12]:
days = driver.find_elements(By.CLASS_NAME, "TableBaseWrapper")

In [8]:
# day
days[0].find_element(By.TAG_NAME, "h4").text

'Wednesday, January 3, 2024'

In [13]:
# img team
days[0].find_element(By.TAG_NAME, "img").get_attribute("src")

'https://sports.cbsimg.net/fly/images/nfl/logos/team/404.svg'

In [11]:
# team
days[0].find_element(By.CLASS_NAME, "TeamName").text

'ARI'

In [13]:
# player
days[0].find_element(By.CLASS_NAME, "CellPlayerName--short").text

'V. Dimukeje'

In [14]:
# injury location
tr1 = days[0].find_elements(By.TAG_NAME, "tr")[1]
tr1

<selenium.webdriver.remote.webelement.WebElement (session="8561462a677efe1efb69536a49c46dfa", element="F11B89B731B04BE599096AE9F42244A6_element_167")>

In [44]:
injury_location = tr1.find_elements(By.TAG_NAME, "td")[3].text

In [15]:
tr1.find_element(By.TAG_NAME, "img").get_attribute("src")

'https://sports.cbsimg.net/fly/images/nfl/logos/team/404.svg'

In [16]:
tr1.find_element(By.CLASS_NAME, "TeamLogo-image").get_attribute("src")

'https://sports.cbsimg.net/fly/images/nfl/logos/team/404.svg'

In [58]:
player_tr = tr1.find_elements(By.TAG_NAME, "td")[1].text

'Victor Dimukeje'

In [61]:
team_tr = tr1.find_elements(By.TAG_NAME, "td")[0].text
team_tr

'ARI'

In [ ]:
datas = {}
for day in range(0, len(days)):
    time.sleep(1)
    # day
    journey = days[day].find_element(By.TAG_NAME, "h4").text
    # tr per day
    trs = days[day].find_elements(By.TAG_NAME, "tr")
    for tr in range(1, len(trs)):
        time.sleep(1)
        # img team
        img_team = trs[tr].find_element(By.TAG_NAME, "img").get_attribute("src")
        # team
        team = trs[tr].find_element(By.CLASS_NAME, "TeamName").text
        # player
        player = trs[tr].find_element(By.CLASS_NAME, "CellPlayerName--short").text
        # injury location
        injury_location = trs[tr].find_elements(By.TAG_NAME, "td")[3].text
        # create dict
        datas[str(random.randint(10000, 1000000))] = {
            'player': player,
            'img_team': img_team,
            'team': team,
            'injury_location': injury_location,
            'injury_date': journey
        }

In [57]:
tr1.find_element(By.CLASS_NAME, "CellPlayerName--short").text

''

In [9]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import sqlalchemy as db
import random
import time

def scraping_injuries(theme='nba', n_days=3):
    
    # 1  Connexion à la base de données
    database = DataBase('Scraping_sport_injuries')

    # 2 Création d'une table
    if 'players-injured' not in database.table:
        database.create_table(
            'players-injured',
            player=db.String,
            sport=db.String,
            img_team=db.String,
            team=db.String,
            injury_location=db.String,
            injury_date=db.String,
        )
    driver = Chrome()
    driver.get(f"https://www.cbssports.com/{theme}/injuries/daily/")
    # reject cookies
    driver.find_element(By.ID, "onetrust-reject-all-handler").click()
    days = driver.find_elements(By.CLASS_NAME, "TableBaseWrapper")
    datas = {}
    for day in range(0, n_days):
        time.sleep(1)
        # day
        journey = days[day].find_element(By.TAG_NAME, "h4").text
        # tr per day
        trs = days[day].find_elements(By.TAG_NAME, "tr")
        for tr in range(1, len(trs)):
            time.sleep(1)
            # img team
            img_team = trs[tr].find_element(By.CLASS_NAME, "TeamLogo-image").get_attribute("src")
            # team
            team = trs[tr].find_elements(By.TAG_NAME, "td")[0].text
            # player
            player = trs[tr].find_elements(By.TAG_NAME, "td")[1].text
            # injury location
            injury_location = trs[tr].find_elements(By.TAG_NAME, "td")[3].text
            if theme == 'nba':
                sport = 'basketball'
            elif theme == 'nfl':
                sport = 'football'
            else:
                sport = 'hockey'
            # create dict
            # add row in db
            if player not in database.select_table('players-injured'):        
                database.add_row('players-injured', id_user=db.Integer, name=player, sport=sport, img_team=img_team, team=team, injury_location=injury_location, injury_date=journey)    
            datas[str(random.randint(10000, 1000000))] = {
                'player': player,
                'sport': sport,
                'img_team': img_team,
                'team': team,
                'injury_location': injury_location,
                'injury_date': journey
            }
    driver.quit()
    return datas        


In [12]:
scraping_injuries(theme='nba', n_days=6)

NameError: name 'DataBase' is not defined

In [11]:
driver.quit()

NameError: name 'driver' is not defined

In [ ]:
# nba nhl nfl mlb

## Enregistrement en base de données

In [21]:
# Gestion d'une base de données
import sqlalchemy as db

class DataBase():
    def __init__(self, name_database='database'):
        self.name = name_database
        self.url = f"sqlite:///{name_database}.db"
        self.engine = db.create_engine(self.url)
        self.connection = self.engine.connect()
        self.metadata = db.MetaData()
        self.table = self.engine.table_names()

    def create_table(self, name_table, **kwargs):
        colums = [db.Column(k, v, primary_key = True) if 'id_' in k else db.Column(k, v) for k,v in kwargs.items()]
        db.Table(name_table, self.metadata, *colums)
        self.metadata.create_all(self.engine)
        print(f"Table : '{name_table}' are created succesfully")

    def read_table(self, name_table, return_keys=False):
        table = db.Table(name_table, self.metadata, autoload=True, autoload_with=self.engine)
        if return_keys:table.columns.keys()
        else : return table


    def add_row(self, name_table, **kwarrgs):
        name_table = self.read_table(name_table)

        stmt = (
            db.insert(name_table).
            values(kwarrgs)
        )
        self.connection.execute(stmt)
        print(f'Row id added')


    def delete_row_by_id(self, table, id_):
        name_table = self.read_table(name_table)

        stmt = (
            db.delete(name_table).
            where(students.c.id_ == id_)
            )
        self.connection.execute(stmt)
        print(f'Row id {id_} deleted')

    def select_table(self, name_table):
        name_table = self.read_table(name_table)
        stm = db.select([name_table])
        return self.connection.execute(stm).fetchall()

In [4]:
# Création d'une base de données
database = DataBase('database_sport_injuries')

C:\Users\antho\AppData\Local\Temp\ipykernel_9788\3108415504.py:11: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  self.table = self.engine.table_names()


In [5]:
# Création d'une table joueurs blessés
database.create_table('players-injured', id_user=db.Integer, name=db.String, sport=db.String, img_team=db.String, team=db.String, injury_location=db.String, injury_date=db.String)

Table : 'players-injured' are created succesfully


In [ ]:
# Ajouter une ligne à la base de données
database.add_row('players-injured', id_user=db.Integer, name=db.String, sport=db.String, img_team=db.String, team=db.String, injury_location=db.String, injury_date=db.String)

## utilisation OpenAI

In [ ]:
#Instalation de la bibliothèque openAI
!pip install openai==0.28.0

In [1]:
import openai
#Placer votre clé API dans la variable ci-dessous
openai.api_key = ""

In [14]:
# Générateur de texte
def prompt_openAI(ask="la plus fréquente en basket"): 
    response = openai.Completion.create(
        engine="davinci",
        prompt="Quelles est la blessure "+ ask +" ?",
        temperature=0.5,
        max_tokens=300)

    return response.choices[0].text

In [16]:
prompt_openAI(ask="la moins fréquente en nhl")

"\n\nAuteur Message\n\nAlfredo\n\n\n\n\n\n\n\n\n\nMessages : 743\n\nDate d'inscription : 18/12/2009\n\nAge : 28\n\nLocalisation : Rimouski\n\nMidget74318/12/200928Rimouski\n\nSujet: Re: Quelle est la blessurela moins fréquente en nhl? Ven 8 Juin - 14:32 Sujet: Re: Quelle est la blessurela moins fréquente en nhl?Ven 8 Juin - 14:32 c'est la blessure au pied\n\n_________________\n\n_________________\n\nContenu sponsorisé\n\n\n\n\n\n\n\n\n\nSujet: Re: Quelle est la blessurela moins fréquente en nhl? Sujet: Re: Quelle est la blessurela moins fréquente en nhl?"

In [27]:
def prompt2(ask="la moins fréquente en nhl"):
    reponse = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system",
            "content": f"Quelles est la blessure "+ ask +" ?"},
            # {"role": "user",
            # "content": "Voici la liste des actualités à synthétiser :" + actu},
        ],
        max_tokens=100,
        temperature=0.9,
    )

    return reponse['choices'][0]['message']["content"]

In [28]:
prompt2(ask="la moins fréquente en nfl")

"La blessure la moins fréquente en NFL est la rupture du tendon d'Achille. C'est une blessure rare qui se produit lorsqu'il y a une déchirure complète du tendon qui relie le muscle du mollet à l'os du talon. La plupart du temps, cette blessure survient lors d'un mouvement brusque ou d'une contraction soudaine du muscle du mollet. Bien que cette blessure soit rare,"

In [ ]:
# fonction openAI
def prompt_openAI(prompt):
    openai.api_key = ""
    reponse = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system",
            "content": f"Quelles est la blessure "+ prompt +" ?"},
        ],
        max_tokens=100,
        temperature=0.9,
    )

    return reponse['choices'][0]['message']["content"]